In [1]:
import os
import requests
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Set the desired number of images to download
num_images = 1000

# Create a directory to store the images
os.makedirs("nsfw_data", exist_ok=True)

# Define the URLs of the NSFW images dataset
urls = [
    "https://github.com/alex000kim/nsfw_data_scraper/raw/master/data/xxx",
    "https://github.com/alex000kim/nsfw_data_scraper/raw/master/data/hentai",
    "https://github.com/alex000kim/nsfw_data_scraper/raw/master/data/drawn_hentai",
    "https://github.com/alex000kim/nsfw_data_scraper/raw/master/data/neutral",
    "https://github.com/alex000kim/nsfw_data_scraper/raw/master/data/porn",
    "https://github.com/alex000kim/nsfw_data_scraper/raw/master/data/sexy"
]

# Download the NSFW images
for url in urls:
    file_name = url.split("/")[-1]
    file_path = os.path.join("nsfw_data", file_name)
    response = requests.get(url)
    with open(file_path, "wb") as f:
        f.write(response.content)


In [2]:


# Set the paths for the downloaded images
data_dir = "nsfw_data"
train_dir = "nsfw_data/train"
validation_dir = "nsfw_data/validation"
test_dir = "nsfw_data/test"

# Create train, validation, and test directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Move images to the train, validation, and test directories
for category in os.listdir(data_dir):
    category_dir = os.path.join(data_dir, category)
    if os.path.isdir(category_dir):
        file_names = os.listdir(category_dir)

        # Skip categories with no images
        if len(file_names) == 0:
            continue

        random.shuffle(file_names)
        train_split = int(0.7 * len(file_names))
        validation_split = int(0.2 * len(file_names))

        for file_name in file_names[:train_split]:
            src = os.path.join(category_dir, file_name)
            dst = os.path.join(train_dir, category, file_name)
            shutil.copyfile(src, dst)

        for file_name in file_names[train_split:validation_split]:
            src = os.path.join(category_dir, file_name)
            dst = os.path.join(validation_dir, category, file_name)
            shutil.copyfile(src, dst)

        for file_name in file_names[validation_split:]:
            src = os.path.join(category_dir, file_name)
            dst = os.path.join(test_dir, category, file_name)
            shutil.copyfile(src, dst)

            

# Set the image dimensions and other constants
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
NUM_EPOCHS = 10

# Create data generators for training, validation, and testing

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode='binary',
    shuffle=False
)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [3]:
# Check train_dir
if os.path.exists(train_dir):
    print(f"train_dir exists: {train_dir}")
else:
    print(f"train_dir does not exist: {train_dir}")

# Check validation_dir
if os.path.exists(validation_dir):
    print(f"validation_dir exists: {validation_dir}")
else:
    print(f"validation_dir does not exist: {validation_dir}")

# Check test_dir
if os.path.exists(test_dir):
    print(f"test_dir exists: {test_dir}")
else:
    print(f"test_dir does not exist: {test_dir}")


train_dir exists: nsfw_data/train
validation_dir exists: nsfw_data/validation
test_dir exists: nsfw_data/test


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Set the image dimensions and other constants
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
NUM_EPOCHS = 10

# Create data generators for training, validation, and testing
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode='binary',
    shuffle=False
)

# Create the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // BATCH_SIZE,
    epochs=NUM_EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // BATCH_SIZE
)

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

# Save the model
model.save('nsfw_model.h5')


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


ValueError: Asked to retrieve element 0, but the Sequence has length 0